<h1 style="text-align: center; font-size: 3em">Monte Carlo</h1>

<h1>Menu</h1>
<ul style="list-style-type: none">
    <li>
        <a href="#montecarlo" style="text-decoration: none">1. Monte Carlo</a>
    </li>
    <li>
        <a href="#bs" style="text-decoration: none">2. Example of the Black-Scholes distribution</a>
        <ul style="list-style-type: none">
            <li>
                <a href="#bs.diffusion" style="text-decoration: none">2.1. Distribution of a future contract</a>
            </li>
            <li>
                <a href="#bs.m_indices" style="text-decoration: none">2.2. Distribution of a future contract for several indices</a>
            </li>
            <li>
                <a href="#bs.vectorisation" style="text-decoration: none">2.3. Computation vectorization</a>
            </li>
            <li>
                <a href="#bs.vectorisation_m_indices" style="text-decoration: none">2.4. Distribution of several futures contracts for several indices</a>
            </li>
        </ul>
    </li>
</ul>

<a id="montecarlo" style="text-decoration: none; color: black; font-size: 2em">1. Monte Carlo</a>

Unique MonteCarlo class that takes as an entry:

<ul>
    <li>A dynamic (Black Scholes, Merton, Bachelor...)</li>
    <li>An option (Simple option, Basket option, Asian option...)</li>
    <li>A number of simulations</li>
</ul>
This class contains a single function: $values$, which returns the $price$, $delta$ and $gamma$ of any option with any dynamic. 

In addition, for each value, a half confidence interval at $95\%$ is calculated to ensure their accuracy.

```python
def values(self, fd1, fd2, relative=True):

    prix, prix_2 = 0, 0
    delta, delta_2 = np.zeros(self.dynamic.size_), np.zeros(self.dynamic.size_)
    gamma, gamma_2 = np.zeros(self.dynamic.size_), np.zeros(self.dynamic.size_)

    for i in range(self.nb_samples):
        path = self.dynamic.path()  # Path simulation
        payoff = self.option.payoff(path)  # The option must have a payoff method
        prix += payoff
        prix_2 += payoff * payoff

        for d in range(self.dynamic.size_):

            # Path shifting
            increment_path = self.dynamic.shift_price(path, d, fd1)
            decrement_path = self.dynamic.shift_price(path, d, fd2)

            # Payoff for shifted path
            increment_payoff = self.option.payoff(increment_path)
            decrement_payoff = self.option.payoff(decrement_path)

            delta_payoff = increment_payoff - decrement_payoff
            delta[d] += delta_payoff
            delta_2[d] += delta_payoff * delta_payoff

            gamma_payoff = increment_payoff + decrement_payoff - 2 * payoff
            gamma[d] += gamma_payoff
            gamma_2[d] += gamma_payoff * gamma_payoff

    delta_coeffs = [fd1 - fd2]
    gamma_coeffs = [((fd1 - fd2) * 0.5) ** 2]

    if relative:
        delta_coeffs *= (self.dynamic.basket.price() / self.dynamic.basket.weights)
        gamma_coeffs *= (self.dynamic.basket.price() / self.dynamic.basket.weights) ** 2

    return MCResults(self.value_and_ic(prix, prix_2), self.values_and_ics(delta, delta_2, delta_coeffs),
                     self.values_and_ics(gamma, gamma_2, gamma_coeffs))
```

<a id="bs" style="text-decoration: none; color: black; font-size: 2em">2. Example of the Black-Scholes distribution</a>

<a id="bs.diffusion" style="text-decoration: none; color: black; font-size: 2em; margin-left: 50px">2.1. Distribution of a future contract</a>

The valuation of an underlying future contract by the Black-Scholes model is described as follows: 

$$F_t = F_{0} e^{-\sigma^2/2 + \sigma B_t}$$

where $B$ is a standard Brownian motion and $\sigma > 0$ is the volatility of the underlying.

We can therefore discretise the problem on the grid $t_0, t_1, ..., t_n$ as follows:

$$F_{t_{i+1}} = F_{t_i} and^{-(t_{i+1} - t_i) \sigma^2/2 + \sigma \sqrt{t_{i+1} - t_i} G_{i+1}}$$

with $(G_i)_{i \geqslant 1}$ a sequence of random numbers of normal law.

<h3>Evaluation de l'option

<ul style="list-style-type: none">
    <li>$\phi$: Option payoff</li>
    <li>$T$: Option maturity</li>
    <li>$M$: Number of Monte Carlo simulations</li>
    <li>$F_0$: Price of the underlying future at 0.</li>
    <li>$F$: Vector of the simulated path. </li>
    <li>$df$: Discount factor between 0 and T (i.e. $e^{-rT}$)</li>
</ul>

An option on a single underlying futures contract produces the following values:
$$\left \{
    \begin{array}{r c l}
        P_0 & = & df \frac{\phi(F)}{M} \\\ 
        \Delta & = & \frac{\phi((1+h)F) - \phi((1-h)F)}{2 M F_0 h} \\
        \Gamma & = & df \frac{\phi((1+h)F) + \phi((1-h)F) - 2 \phi(F)}{M F_0^2 h}
    \end{array}
\right.$$

<a id="bs.m_indices" style="text-decoration: none; color: black; font-size: 2em; margin-left: 50px">2.2. Distribution of a future contract for several indices</a>

Consider m assets, each evolving according to a Black Scholes model of dimension 1 and correlated with each other. 
The dynamic F of these m assets is written:

$$F_{t, d} = F_{0, d} e^{-\sigma_d^2/2 + \sigma_d B_{t,d}}$$

where $B = (B_1, ..., ..., B_m)^T$ a vector of correlated standard Brownian motions.

Let the sequence $W = (W_1, ..., W_d)^T$, then we have equality in law $(B_t)_{t \geqslant 0} = (L.W_t)_{t \geqslant 0}$
where L is the Cholesky factor of the correlation matrix (which must be positively defined). 

The equation can then be rewritten as follows:
$$F_{t, d} = F_{0, d} e^{-\sigma_d^2/2 + \sigma_d L_d W_t}$$$

where $L_d$ is the line d of the matrix L.

We can therefore deduce the following discretized equation:

$$F_{t_{i+1}, d} = F_{t_i, d} e^{-(t_{i+1} - t_i) \sigma_d^2/2 + \sigma_d \sqrt{t_{i+1} - t_i} L_d G_{i+1}} $$

with $(G_i)_{i \geqslant 1}$ a sequence of Gaussian vectors.

<h3>Option valuation

<ul style="list-style-type: none">
    <li>$n$: Number of discretization steps</li>
    <li>$m$: Number of indices</li>
    <li>$w_d$: Weight of the underlying d in the basket</li>
    <li>$F_{0, d}$: Price of the underlying future d at time 0</li>
    <li>$F$: Matrix of the simulated path. $(n+1, m)$</li>
    <li>$fd(d, h) = \begin{pmatrix} 1 & 0 & 0 \\\ 0 & 1 + h & 0 \\ 0 & 0 & 1 \end{pmatrix}$ relative bump of h for index d</li>
</ul>

An option on m indices each with an underlying futures contract produces the following values:
$$\left \{
    \begin{array}{r c l}
        P_0 & = & df \frac{\phi(F)}{M} \\\ 
        \Delta_d & = & \frac{\phi(F.fd(d, h)) - \phi(F.fd(d, -h))}{2MF_{0, d}h} \Rightarrow
        \Delta = \sum_{d=1}^{m} w_d \Delta_d\\
        \Gamma_d & = & df \frac{\phi(F.fd(d, h)) + \phi(F.fd(d, -h)) - 2 \phi(F)}{M F_{0, d}^2 h} \Rightarrow  
        \Gamma = \sum_{d=1}^{m} w_d \Gamma_d
    \end{array}
\right. $$.

<a id="bs.vectorisation" style="text-decoration: none; color: black; font-size: 2em; margin-left: 50px">2.3. Computation vectorization</a>

The python language is not as powerful as C or C++ languages, so use the numpy library written in C significantly reduces the calculation time.

<ul style="list-style-type: none">
    <li>$w$: Weight vector</li>
    <li>$F_0$: Future price vector at the horizon date $(m,)$</li>
    <li>$\sigma$: Contract volatility vector $(m,)$</li>
    <li>$Ts$: Discretization vector $(n+1, )$</li>
    <li>$L$: Cholesky matrix derived from the correlation matrix between the $(m, m)$</li> indices
    <li>$G$: Normal random number matrix $(m, n+1)$</li>
    <li>$\sum$: $\begin{pmatrix}
        0 & 1 & 1 \\
        0 & 0 & 1 \\
        0 & 0 & 0
        \end{pmatrix}$ size $(n+1, n+1)$</li>
    <li>. Matrix product</li>
    <li>$\times$: Term to term product</li>
    <li>$⊗$: Tensor product</li>
</ul>

F the simulated path matrix for $m$ number of indices and $n$ number of simulation steps is written:
$$F = F_0 \times e^{(-0.5 \sigma^2 ⊗ Ts + (\sigma ⊗ \sqrt{Ts}) \times (L.G))$$

Allowing us to write the price, delta and gamma by index:
$$\left \{
    \begin{array}{r c l}
        P & = & \frac{\phi(F)}{M} \\\ 
        \Delta_d & = & \frac{\phi(fd(d, h).F) - \phi(fd(d, -h).F)}{2MF_{0, d}h} \\
        \Gamma_d & = & \frac{\phi(fd(d, h).F) + \phi(fd(d, -h).F) - 2 \phi(F)}{M F_{0, d}^2 h}
    \end{array}
\right. $$.

<a id="bs.vectorisation_m_indices" style="text-decoration: none; color: black; font-size: 2em; margin-left: 50px">2.4. Distribution of several futures contracts for several indices</a>

However, since the commodities market is affected by futures contracts, the valuation of an Asian option 
on future is different from an Asian option on spot. Indeed, to estimate an Asian option on 
future, all future contracts must be diffused between the horizon date (e. g. 16/03/19) and the date 
of observation (e. g. 01/04/19). We can therefore potentially have m indices that each have p futures contracts and 
want to distribute them on the basis of $n$ discretization steps.

<ul style="list-style-type: none">
    <li>$m$: Number of indices</li>
    <li>$p$: Number of futures contracts</li>
    <li>$n$: Number of discretization steps</li>
    <li>$F_0$: Matrix of future prices at the horizon $(p, m)$</li>
    <li>$\sigma$: Contract volatility vector $(p, m)$</li>
    <li>$Ts$: Matrix of discretization steps $(n)$</li>
    <li>$L$: Cholesky matrix derived from the correlation matrix between the $(m, m)$</li> indices
    <li>$G$: Normal random number matrix $(m, n)$</li>
    <li>$\sum$: $\begin{pmatrix}
        0 & 1 & 1 \\
        0 & 0 & 1 \\
        0 & 0 & 0
        \end{pmatrix}$ size $(n+1, n+1)$</li>
    <li>. Matrix product</li>
    <li>$\times$: Term term product</li>
    <li>$⊗$: Tensor product</li>
</ul>

F the simulated path matrix for $p$ futures contract, $m$ indices and $n$ simulation steps is written:
$$F = F_0 ⊗ e^{(-0.5 \sigma^2 ⊗ Ts + (\sigma ⊗ \sqrt{Ts}) \times (L.G)). \sum}$$

Allowing us to write the price, delta and gamma by index:
$$\left \{
    \begin{array}{r c l}
        P & = & \frac{\phi(F)}{M} \\\ 
        \Delta_d & = & \frac{\phi(fd(d, h) ⊗ F) - \phi(fd(d, -h) ⊗ F)}{2MF_{0, d}h} \\
        \Gamma_d & = & \frac{\phi(fd(d, h) ⊗ F) + \phi(fd(d, -h) ⊗ F) - 2 \phi(F)}{M F_{0, d}^2 h}
    \end{array}
\right. $$.